In [ ]:
# PWM test in Programmable Logic (PL)

In [1]:
!pwd

/root/jupyter_notebooks/pynq-pwm/send-kr260-pwm-test


In [2]:
!ls

design_1_wrapper.bit  design_1_wrapper.hwh  PWM-test-PCB.ipynb


In [3]:
from pynq import Overlay
from pynq.lib import AxiGPIO

ol = Overlay("/root/jupyter_notebooks/pynq-pwm/design_1_wrapper.bit")


In [4]:
gpio_0_ip = ol.ip_dict['axi_gpio_0']
gpio_out = AxiGPIO(gpio_0_ip).channel1

mask = 0xffffffff
gpio_out.write(0x1,mask)

In [5]:

# utility functions for bit manipulation
def set_bit(value, bit):
    return value | (1 << bit)

def clear_bit(value, bit):
    return value & ~(1 << bit)

def get_bit(value, bit):
    return (value >> bit) & 1

motor_B2 = ol.axi_timer_0
motor_B1 = ol.axi_timer_1
motor_A2 = ol.axi_timer_2
motor_A1 = ol.axi_timer_3

# extract register addresses (will be the same for every Axi Timer)
TCSR0 = ol.ip_dict['axi_timer_0']['registers']['TCSR0']
TCSR1 = ol.ip_dict['axi_timer_0']['registers']['TCSR1']
TCSR0_address = TCSR0['address_offset']
TCSR1_address = TCSR1['address_offset']
TCSR0_register = TCSR0['fields'] # bit_offset for address
TCSR1_register = TCSR1['fields']
TLR0 = ol.ip_dict['axi_timer_0']['registers']['TLR0']
TLR1 = ol.ip_dict['axi_timer_0']['registers']['TLR1']
TLR0_address = TLR0['address_offset']
TLR1_address = TLR1['address_offset']

# create the configuration values for the control register
temp_val_0 = 0
temp_val_1 = 0

# The PWMA0 bit in TCSR0 and PWMB0 bit in TCSR1 must be set to 1 to enable PWM mode.
temp_val_0 = set_bit(temp_val_0, TCSR0_register['PWMA0']['bit_offset'])
temp_val_1 = set_bit(temp_val_1, TCSR1_register['PWMA1']['bit_offset'])

# The GenerateOut signals must be enabled in the TCSR (bit GENT set to 1). The PWM0
# signal is generated from the GenerateOut signals of Timer 0 and Timer 1, so these
# signals must be enabled in both timer/counters
temp_val_0 = set_bit(temp_val_0, TCSR0_register['GENT0']['bit_offset'])
temp_val_1 = set_bit(temp_val_1, TCSR1_register['GENT1']['bit_offset'])

# The counter can be set to count up or down. UDT
temp_val_0 = set_bit(temp_val_0, TCSR0_register['UDT0']['bit_offset'])
temp_val_1 = set_bit(temp_val_1, TCSR1_register['UDT1']['bit_offset'])

# set Autoreload (ARHT0 = 1)
temp_val_0 = set_bit(temp_val_0, TCSR0_register['ARHT0']['bit_offset'])
temp_val_1 = set_bit(temp_val_1, TCSR1_register['ARHT1']['bit_offset'])

# enable timer (ENT0 = 1)
temp_val_0 = set_bit(temp_val_0, TCSR0_register['ENT0']['bit_offset'])
temp_val_1 = set_bit(temp_val_1, TCSR1_register['ENT1']['bit_offset'])

# here you must see "0b1010010110" twice
print(bin(temp_val_0))
print(bin(temp_val_1))



0b1010010110
0b1010010110


In [6]:
# B_motor_pwm

def set_motor_B_pwm(duty_cycle, direction):
    _period_ = 20000  # 50Hz, 20ms
    _pulse_ = duty_cycle  # 0-100
    period = int((_period_ & 0x0ffff) * 100)
    pulse = int((_pulse_ & 0x07f) * period / 100)

    print(f"period 20ms: {period}")
    print(f"pulse {duty_cycle}%: {pulse}")
    print(f"direction: {direction}")

    motor_B1.write(TCSR0['address_offset'], temp_val_0)
    motor_B1.write(TCSR1['address_offset'], temp_val_1)
    motor_B1.write(TLR0['address_offset'], period)

    motor_B2.write(TCSR0['address_offset'], temp_val_0)
    motor_B2.write(TCSR1['address_offset'], temp_val_1)
    motor_B2.write(TLR0['address_offset'], period)

    # direction
    if direction == 'forward':
        motor_B1.write(TLR1['address_offset'], pulse)
        motor_B2.write(TLR1['address_offset'], 0)
    elif direction == 'reverse':
        motor_B1.write(TLR1['address_offset'], 0)
        motor_B2.write(TLR1['address_offset'], pulse)
    elif direction == 'coast':
        motor_B1.write(TLR1['address_offset'], 0)
        motor_B2.write(TLR1['address_offset'], 0)
    elif direction == 'break':
        motor_B1.write(TLR1['address_offset'], 100)
        motor_B2.write(TLR1['address_offset'], 100)   
    else:
        print("Invalid direction. Please use 'forward' or 'reverse'or 'coast' or 'break'.")


In [7]:
import time

# B_motor_forward_test
set_motor_B_pwm(10, 'forward')
time.sleep(3)
set_motor_B_pwm(50, 'forward')
time.sleep(3)
set_motor_B_pwm(99, 'forward')
time.sleep(3)

set_motor_B_pwm(0, 'coast')


period 20ms: 2000000
pulse 10%: 200000
direction: forward
period 20ms: 2000000
pulse 50%: 1000000
direction: forward
period 20ms: 2000000
pulse 99%: 1980000
direction: forward
period 20ms: 2000000
pulse 0%: 0
direction: coast


In [8]:
# B_motor_reverse_test
set_motor_B_pwm(10, 'reverse')
time.sleep(3)
set_motor_B_pwm(50, 'reverse')
time.sleep(3)
set_motor_B_pwm(99, 'reverse')
time.sleep(3)

set_motor_B_pwm(0, 'coast')


period 20ms: 2000000
pulse 10%: 200000
direction: reverse
period 20ms: 2000000
pulse 50%: 1000000
direction: reverse
period 20ms: 2000000
pulse 99%: 1980000
direction: reverse
period 20ms: 2000000
pulse 0%: 0
direction: coast


In [9]:
mask = 0xffffffff
gpio_out.write(0x0,mask)